In [11]:
from transformers import  AutoTokenizer, AutoModelForSeq2SeqLM
from dimweb_persona_bot.dataloaders.seq2seq_samplers.seq2seq_samplers_hypothesis_2 import H2Seq2SeqInferencePersonaSampleV1
from dimweb_persona_bot.hyperparameters.causal_modeling_hyperparameters import H2PersonaChatHyperparametersV1
import torch

In [6]:
history = [1, 2, 3, 4, 5, 6]

[(history[i], history[i+1]) for i in range(0, len(history), 2) ]

[(1, 2), (3, 4), (5, 6)]

In [9]:
" | ".join([
            "i'm also a graduate student .",
            "i enjoy reading journals and guides related to psychology .",
            "my parents taught me survival skills .",
            "i walk dogs for a living .",
        ])

"i'm also a graduate student . | i enjoy reading journals and guides related to psychology . | my parents taught me survival skills . | i walk dogs for a living ."

In [8]:
"".split(" | ")

['']

In [12]:
model_path = "dim/persona_bot_2_28akcwik"
device = "cuda"

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

hyperparameters = H2PersonaChatHyperparametersV1(
	model_name="facebook/bart-base",
	model_architecture="seq2seq",
	chat_history_pair_length=7,
	persona_max_length=14, 
	chat_max_length=25,
)

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [101]:
sample = H2Seq2SeqInferencePersonaSampleV1(
	tokenizer=tokenizer,
	hyperparameters=hyperparameters,
	dataset_sample={
		"persona": [
      		'I like chocolate ice cream.', 
        	"Sometimes I feel lonely.", 
        	"I like to play video games."
        ],
		"history": [
    	  	"Hi, do you like ice cream?", 
       		'i do like ice cream but i prefer chocolate ice cream',
			"I feel lonely",
			'i feel lonely when i play video games',
			"Do you wanna play with me? I think we can have fun together.",
			'of course. what else do you like?',
			"I like driving simulator games. What about you?",
			'i like driving simulator games too',
			"Let's play together.",
			'okay. what else do you like?',
			"When so you wanna play?",
    ]
	}
).get_sample()

for key in sample.keys():
    sample[key] = torch.tensor(sample[key]).unsqueeze(0).to(device)
    
# model(**sample)
answer = model.generate(**sample, 
	# do_sample=True, 
    # top_p=0.95,
    penalty_alpha=0.1, 
    top_k=10
)
tokenizer.batch_decode(answer, skip_special_tokens=True,)

['anytime. what is your favorite?']

In [108]:
from dimweb_persona_bot.inference.seq2seq_bots import DialogBotV2

bot2 = DialogBotV2(model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=[
        'Hi, how are you doing?',
        "i'm doing well. how are you?",
        "I'm fixing a bug right now",
        "oh wow that's interesting. what bug are you fixing?",
	],
    persona=[
        "I'm a junior frontend developer.", 
        'I like racing games.', 
        'Sometimes I write code for fun.', 
        "I'm a computer science fresher."
	]
)

In [111]:
print("\n".join(["'Hi, how are you doing?'", "i'm doing well, how are you?", "I'm fixing a bug right now", "what bug? i'm a developer.", 'Button sometimes turns red, sometimes disappears', "that's interesting. i'm a computer science major.", 'Can you help me please with it?', 'sure. what do you do for fun?', "I like to play in Dirt2. It's racing game", "that's a very popular game.", 'Do you like it?', 'i do. it is very fun.']))

'Hi, how are you doing?'
i'm doing well, how are you?
I'm fixing a bug right now
what bug? i'm a developer.
Button sometimes turns red, sometimes disappears
that's interesting. i'm a computer science major.
Can you help me please with it?
sure. what do you do for fun?
I like to play in Dirt2. It's racing game
that's a very popular game.
Do you like it?
i do. it is very fun.


In [113]:
from dimweb_persona_bot.dataloaders.persona_chat_dataloaders import PersonaChatDatasetV1
dataset = PersonaChatDatasetV1(input_dataset_path="./datasets/persona_chat/valid.json")

dataset[104]

{'persona': ["i'm also a graduate student .",
  'i enjoy reading journals and guides related to psychology .',
  'my parents taught me survival skills .',
  'i walk dogs for a living .'],
 'history': ['hey how are you today',
  "i'm busy studying psychology for my graduate school class . you ?",
  'i have retired and now spend my time as a pro gambler',
  'sounds cool ! i could be your dog walker when you are busy .',
  'well being retired i have a lot of spare time either in a casino or outdoors',
  'i love the outdoors . i can survive in the wilderness for weeks',
  'yeah me to just got back from two weeks in the bush',
  'did your parents teach you to survive like mine did ?',
  'my dad did he has his own outdoor show its possible you have seen it',
  'i may have . what is it called ?',
  'jim shockey he has survived in places all over the world',
  'i love that show ! i watch it between studying .',
  "yeah didn't get to see him much as a child",
  'that must have been hard . my pa

In [109]:
response = bot2.single_chat(
	'That is cool. What do you do on this work?',
)
print(response)
print(bot2.persona)
print(bot2.history)

i'm a senior frontend developer.
['Sometimes I write code for fun.', "I'm a junior frontend developer.", "I'm a computer science fresher.", 'I like racing games.']
['Hi, how are you doing?', "i'm doing great. just got home from a long day of work.", 'Where do you work?', 'i work in the computer sciences department at the university of louisiana.']


In [53]:
bot2.history

['Hi, how are you doing?',
 "i'm doing well. how are you?",
 "I'am fixing a bug right now",
 "oh wow that's interesting. what bug are you fixing?",
 "i'm fixing a computer that needs fixed",
 "oh wow that's pretty cool. what do you do for fun?"]

In [35]:
bot2.start_chat()

Bot: i'm doing well. how are you?
Bot: oh wow that's interesting. what bug are you fixing?
Bot: i like red hot chilli peppers


In [36]:
bot2.history

['Hi, how are you doing?',
 "i'm doing well. how are you?",
 "I'am fixing a bug right now",
 "oh wow that's interesting. what bug are you fixing?",
 "Oh, It's disgusting bug. The button on website sometimes is black and sometimes is red. What do you think?",
 'i like red hot chilli peppers']

### russian bot

In [3]:
from dimweb_persona_bot.inference.seq2seq_bots import DialogBotV1, DialogBotV2

In [10]:
model_path = "./models/28akcwik/6/"
device = "cuda"

model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(model_path)

hyperparameters = H2PersonaChatHyperparametersV1(
    chat_history_pair_length=7,
    max_response_length=512,
    persona_max_length=14,
    chat_max_length=25,
)

NameError: name 'AutoModelForSeq2SeqLM' is not defined

In [4]:
bot2 = DialogBotV2(model=model,
    tokenizer=tokenizer,
    hyperparameters=hyperparameters,
    history=[
        
	],
    persona=[
        "Я студент 4 курса.",
        "Люблю иногда бухать по пятницам.",
        "У меня нет девушки.",
        "Люблю писать код.",
	]
)

In [5]:
bot2.start_chat()

Bot: Привет, я студент 4 курса. А ты?
Bot: А что ты делаешь в свободное время?
Bot: Я пишу код. А ты?
Bot: В свободное время я играю в футбол. А ты?


### own russian dataset v1

In [1]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from dimweb_persona_bot.hyperparameters.causal_modeling_hyperparameters import H2PersonaChatHyperparametersV1
from datasets import load_dataset, load_from_disk

dataset = load_from_disk("./datasets/ru_dialog_dataset_v1_mbart-50/")

In [2]:
path = "./models/2uuglxhm/checkpoint-140000"
model = AutoModelForSeq2SeqLM.from_pretrained(path)
model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(path)

In [3]:
dataset['train'][0]

{'context': 'Привет привет ! Как настроение ? Чем занимаешься ?',
 'knowledge': 'Я фотограф. У меня есть собака. Мне нравится фотографировать. Я люблю шоколад. Я слушаю поп-рок.',
 'dataset_source': 'RUPersonaChatDatasetV3',
 'label': 'Привет! Я фотограф. А ты?',
 'sample_id': 'RUPersonaChatDatasetV3_0_1',
 'input_ids': [250021,
  1509,
  38335,
  5,
  447,
  3925,
  3549,
  59095,
  59,
  5,
  24126,
  87813,
  38335,
  16833,
  5,
  1509,
  81880,
  112951,
  5,
  1509,
  28203,
  743,
  62659,
  9,
  17329,
  5,
  1813,
  18454,
  146038,
  38,
  5187,
  124441,
  32,
  94321,
  33309,
  132797,
  32,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'labels': [250021, 1813, 18454, 38, 1509, 38335, 5, 536, 4789, 32, 2]}

In [5]:
input_ids = tokenizer(
	"Я фотограф. У меня есть собака. Мне нравится фотографировать. Я люблю шоколад. Я слушаю поп-рок. Привет привет ! Как настроение ? Чем занимаешься ?", 
    return_tensors="pt",     
    ).input_ids.to("cuda")

result = model.generate(
	input_ids,
    penalty_alpha=0.15,
    top_k=20,
)

tokenizer.batch_decode(result, skip_special_tokens=True)

['Привет! Отлично, а у тебя?']

In [6]:
from dimweb_persona_bot.inference.seq2seq_bots import DialogBotV3
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

path = "./models/2uuglxhm/checkpoint-160000"
model = AutoModelForSeq2SeqLM.from_pretrained(path)
model.to('cuda')
tokenizer = AutoTokenizer.from_pretrained(path)

In [11]:
ru_bot = DialogBotV3(
    model=model,
    tokenizer=tokenizer,
    history=[
		"Привет как дела ?",
		'Привет, отлично, как твои?',
		"Хорошо, расскажи сказку про белого бычка.",
		# 'Какую сказку?',
		# "Сказку про белого бычка",
		# 'Какую сказку?',
		# "Ты понимаешь что я говорю ?",
		# 'Да, я понимаю, что ты говоришь.',
		# "Какую компьютерную игру ты любишь?",
	]
)

ru_bot.next_response()

Привет как дела? <sep> Привет, отлично, как твои? <sep> Хорошо, расскажи сказку про белого бычка.


'Какую?'

In [6]:
tokenizer.batch_decode(
    tokenizer.batch_encode_plus(
		[
			"Привет как дела ?",
			'Привет, отлично, как твои?',
			"Хорошо, расскажи смешную сказку про белого бычка",
		],
		add_special_tokens=False,
		max_length=200,
	)['input_ids'],
    add_special_tokens=False,
)

['Привет как дела?',
 'Привет, отлично, как твои?',
 'Хорошо, расскажи смешную сказку про белого бычка',
 'Какую сказку?',
 'про белого бычка']

## independence code for seq2seq persona generation

In [1]:
from typing import List, TypedDict
from dataclasses import dataclass
from itertools import chain

from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import torch


@dataclass
class H2PersonaChatHyperparametersV1:
    """
    chat_history_pair_length: int - количество пар диалога с конца
    """

    model_name: str = "facebook/bart-base"
    chat_history_pair_length: int = 7

    persona_max_length: int = 14
    chat_max_length: int = 25

    debug_status: int = 0


class PersonaChatDatasetSampleV1(TypedDict):
    """
    persona: List[str] - набор предложений фактов персоны
    history: List[str] - набор предложений истории переписки
    """

    persona: List[str]
    history: List[str]
    sample_id: str


class H2Seq2SeqInferenceSampleDictV1(TypedDict):
    input_ids: List[int]
    attention_mask: List[int]


class H2Seq2SeqInferenceSampleDictV2(TypedDict):
    input_ids: torch.Tensor
    attention_mask: torch.Tensor


def flat_list(list_of_lists: List[List]) -> List:
    return list(chain.from_iterable(list_of_lists))


class H2Seq2SeqInferencePersonaSampleV1:
    def __init__(
        self,
        dataset_sample: PersonaChatDatasetSampleV1,
        tokenizer: AutoTokenizer,
        hyperparameters: H2PersonaChatHyperparametersV1,
    ) -> None:
        self.dataset_sample = dataset_sample
        self.tokenizer = tokenizer
        self.hyperparameters = hyperparameters

    def add_spaces_after(
        self,
        items: List[str],
    ) -> List[str]:
        items = [item + " " for item in items]
        return items

    @property
    def bos_token_id(self):
        if "t5" in self.hyperparameters.model_name:
            return []

        if self.tokenizer.bos_token_id is None:
            return []

        return [self.tokenizer.bos_token_id]

    @property
    def eos_token_id(self):
        if self.tokenizer.eos_token_id is None:
            return []

        return [self.tokenizer.eos_token_id]

    def add_sep_beetween(self, items: List[str], sep=" EOS ") -> List[str]:
        for i in range(1, len(items)):
            items[i] = sep + items[i]

        return items

    def add_spaces_between(self, items: List[str]) -> List[str]:
        items = self.add_spaces_after(items)
        items[-1] = items[-1].strip()
        return items

    def get_sample(self) -> H2Seq2SeqInferenceSampleDictV1:

        dialog_history = self.dataset_sample["history"]
        dialog_history = dialog_history[-self.hyperparameters.chat_history_pair_length * 2 - 1 :]
        dialog_history = self.add_sep_beetween(dialog_history)

        persona = self.dataset_sample["persona"]
        persona = self.add_sep_beetween(
            persona,
            sep=" ",
        )

        KNOWLEDGE_IDS = self.tokenizer.encode(
            " [KNOWLEDGE] ",
            add_special_tokens=False,
        )
        CONTEXT_IDS = self.tokenizer.encode(
            " [CONTEXT]",
            add_special_tokens=False,
        )

        encoded_history = self.tokenizer.batch_encode_plus(
            dialog_history,
            add_special_tokens=False,
            truncation=True,
            max_length=self.hyperparameters.chat_max_length,
        )
        encoded_history = flat_list(encoded_history["input_ids"])

        encoded_persona = self.tokenizer.batch_encode_plus(
            persona,
            add_special_tokens=False,
            truncation=True,
            max_length=self.hyperparameters.persona_max_length,
        )

        encoded_persona = flat_list(encoded_persona["input_ids"])

        input_ids = [
            *self.bos_token_id,
            *CONTEXT_IDS,
            *encoded_history,
            *KNOWLEDGE_IDS,
            *encoded_persona,
            *self.eos_token_id,
        ]

        attention_mask = [1] * len(input_ids)

        return H2Seq2SeqInferenceSampleDictV1(
            input_ids=input_ids,
            attention_mask=attention_mask,
        )


class DialogBotV1:
    def __init__(
        self,
        model: AutoModelForSeq2SeqLM,
        tokenizer: AutoTokenizer,
        hyperparameters: H2PersonaChatHyperparametersV1,
        history: List[str] = None,
        persona: List[str] = None,
        device: str = "cuda",
        shuffle_persona: bool = True,
    ):
        self.model = model

        self.tokenizer = tokenizer
        self.hyperparameters = hyperparameters
        self.device = device
        self.shuffle_persona = shuffle_persona

        self.debug_status = hyperparameters.debug_status

        if history is None:
            self.history = []
        self.history = history

        if persona is None:
            self.persona = []
        self.persona = persona

    def _get_sample(
        self,
        persona: List[str],
        history: List[str],
    ) -> H2Seq2SeqInferenceSampleDictV1:
        dataset_sample = PersonaChatDatasetSampleV1(
            persona=persona,
            history=history,
        )

        sample = H2Seq2SeqInferencePersonaSampleV1(
            tokenizer=self.tokenizer,
            hyperparameters=self.hyperparameters,
            dataset_sample=dataset_sample,
        )
        sample = sample.get_sample()
        print(self.tokenizer.decode(sample['input_ids']))

        for key in sample.keys():
            sample[key] = torch.tensor(sample[key]).unsqueeze(0).to(self.device)

        return sample

    def next_response(
        self,
        **generation_params,
    ) -> str:
        """
        делает предсказание на основе текущей истории
        и персоны
        """

        sample = self._get_sample(
            persona=self.persona,
            history=self.history,
        )
        answer = self.generate_response(
            sample,
            **generation_params,
        )
        answer = self.tokenizer.batch_decode(
            answer,
            skip_special_tokens=True,
        )
        self.history.append(answer[0])
        return answer[0]

    def generate_response(
        self,
        sample: H2Seq2SeqInferenceSampleDictV1,
        **generation_params,
    ):
        """
        generation_params - https://huggingface.co/docs/transformers/v4.24.0/en/main_classes/text_generation
        """
        with torch.no_grad():
            return self.model.generate(
                **sample,
                **generation_params,
            )

In [2]:
# facebook/nllb-200-distilled-600M
# PRETRAINED_MODEL_NAME_OR_PATH = "./models/23khz8hr/5/"

# facebook/mbart-large-50
PRETRAINED_MODEL_NAME_OR_PATH = "./models/1zbrflmj/3/"

# sberbank-ai/ruT5-base
# PRETRAINED_MODEL_NAME_OR_PATH = "./models/2y5mzb10/6/"

PAIR_DIALOG_HISTORY_LENGTH = 2

# CHAT_MAX_LENGTH for single sentence
CHAT_MAX_LENGTH = 25
# PERSONA_MAX_LENGTH for single sentence
PERSONA_MAX_LENGTH = 19

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSeq2SeqLM.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)
model.to(device)
model.eval()

tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL_NAME_OR_PATH)

if torch.cuda.is_available():
	model.half()

hyperparameters = H2PersonaChatHyperparametersV1(
	chat_history_pair_length=PAIR_DIALOG_HISTORY_LENGTH,
	persona_max_length=PERSONA_MAX_LENGTH,
	chat_max_length=CHAT_MAX_LENGTH,
	model_name=PRETRAINED_MODEL_NAME_OR_PATH,
)


In [4]:

persona = [
 	"Я люблю играть с милыми песиками",
	"Я ненавижу лук и броколли"
]

history = [
	"Привет. Что ты любишь лук?"
]
            
persona_bot = DialogBotV1(
        model=model,
        tokenizer=tokenizer,
        hyperparameters=hyperparameters,
        history=history,
        persona=persona,
        device=device,
    )

GENERATION_PARAMS = {
	"max_new_tokens": 60,
	"penalty_alpha": 0.15,
	"top_k": 10
}
response = persona_bot.next_response(
	**GENERATION_PARAMS,
)

print(response)

[CONTEXT] Привет. Что ты любишь лук? [KNOWLEDGE]  Я люблю играть с милыми песиками Я ненавижу лук и броколли</s>
Привет.


In [6]:
from huggingface_hub import notebook_login

notebook_login()

Token is valid.
Your token has been saved to /home/kosenko/.huggingface/token
Login successful


In [3]:
model.push_to_hub("DeepPavlov/mbart-large-50-ru-persona-chat")
tokenizer.push_to_hub("DeepPavlov/mbart-large-50-ru-persona-chat")

CommitInfo(commit_url='https://huggingface.co/DeepPavlov/mbart-large-50-ru-persona-chat/commit/3598b5ccf26927040669e6fdb04de32c2e0944ee', commit_message='Upload tokenizer', commit_description='', oid='3598b5ccf26927040669e6fdb04de32c2e0944ee', pr_url=None, pr_revision=None, pr_num=None)